In [1]:
import pandas as pd

# Load CSV files
match_results_df = pd.read_csv('csv/wedstrijden_onverwerkd.csv')
goal_events_df = pd.read_csv('csv/doelpunten_onverwerkd.csv')
standings_df = pd.read_csv('csv/klassement_onverwerkd.csv')

goal_events_df = goal_events_df[goal_events_df['valid_goal'] == True]
# Filter of vervang niet-numerieke waarden
match_results_df['doelpunten_thuisploeg'] = pd.to_numeric(match_results_df['doelpunten_thuisploeg'], errors='coerce').fillna(0)
match_results_df['doelpunten_uitploeg'] = pd.to_numeric(match_results_df['doelpunten_uitploeg'], errors='coerce').fillna(0)

# Nu kunt u de conversie naar int veilig uitvoeren
match_results_df['doelpunten_thuisploeg'] = match_results_df['doelpunten_thuisploeg'].astype(int)
match_results_df['doelpunten_uitploeg'] = match_results_df['doelpunten_uitploeg'].astype(int)

In [2]:
# Vind alle verplaatste matches, namelijk de match id zal dezelfde zijn als de vorige
match_results_df['same_as_previous_match'] = match_results_df['id_match'].eq(match_results_df['id_match'].shift())

# Selecteer alleen de rijen waar 'same_as_previous_match' True is
duplicate_matches = match_results_df[match_results_df['same_as_previous_match']]
duplicate_matches.head(30)

,seizoen,speeldag,datum,tijd,id_match,thuisploeg_stamnummer,thuisploeg,uitploeg_stamnummer,uitploeg,doelpunten_thuisploeg,doelpunten_uitploeg,same_as_previous_match
736,2021,22,2022/01/15,16:15:00,3598293,2495,Oud-Heverlee Leuven,25,KV Mechelen,0,0,True
959,2020,12,2020/12/08,17:00:00,3415335,2495,Oud-Heverlee Leuven,216,Royal Excel Moeskroen (-2022),0,0,True


In [3]:
# Bereken goals
goals_home_team = goal_events_df[goal_events_df['goal_team_stamnummer'] == goal_events_df['thuisploeg_stamnummer']].groupby('id_match').size()
goals_away_team = goal_events_df[goal_events_df['goal_team_stamnummer'] == goal_events_df['uitploeg_stamnummer']].groupby('id_match').size()

# Omzetting
goals_home_team_df = goals_home_team.reset_index(name='calculated_doelpunten_thuisploeg')
goals_away_team_df = goals_away_team.reset_index(name='calculated_doelpunten_uitploeg')

# Vervang NaN values in goal kolom met 0
merged_df = pd.merge(match_results_df, goals_home_team_df, on='id_match', how='outer')
merged_df = pd.merge(merged_df, goals_away_team_df, on='id_match', how='outer')
merged_df['calculated_doelpunten_thuisploeg'] = merged_df['calculated_doelpunten_thuisploeg'].fillna(0).astype(int)
merged_df['calculated_doelpunten_uitploeg'] = merged_df['calculated_doelpunten_uitploeg'].fillna(0).astype(int)

# Bereken verschil tussen berekening en resultaat
discrepancies_goals = merged_df[
    (merged_df['calculated_doelpunten_thuisploeg'] != merged_df['doelpunten_thuisploeg']) |
    (merged_df['calculated_doelpunten_uitploeg'] != merged_df['doelpunten_uitploeg'])
]

# Select only relevant columns for display
discrepancies_goals.head(50)

,seizoen,speeldag,datum,tijd,id_match,thuisploeg_stamnummer,thuisploeg,uitploeg_stamnummer,uitploeg,doelpunten_thuisploeg,doelpunten_uitploeg,same_as_previous_match,calculated_doelpunten_thuisploeg,calculated_doelpunten_uitploeg
364,2022,14,2022/10/23,18:30:00,3851182,16,Standard Luik,35,RSC Anderlecht,5,0,False,3,1
388,2022,17,2022/11/12,20:45:00,3851244,29,R Charleroi SC,25,KV Mechelen,0,5,False,1,0
736,2021,22,2022/01/15,16:15:00,3598293,2495,Oud-Heverlee Leuven,25,KV Mechelen,0,0,True,2,2
849,2021,34,2022/04/10,18:30:00,3598351,50,Union Saint-Gilloise,13,Beerschot V.A.,5,0,False,0,0
958,2020,12,2020/11/08,20:45:00,3415335,12,Cercle Brugge,2159,SV Zulte Waregem,1,3,False,3,3
959,2020,12,2020/12/08,17:00:00,3415335,2495,Oud-Heverlee Leuven,216,Royal Excel Moeskroen (-2022),0,0,True,3,3
2004,2016,17,2016/12/04,18:00:00,2732129,29,R Charleroi SC,16,Standard Luik,0,0,False,1,3
2435,2014,11,2014/10/19,14:30:00,2503727,16,Standard Luik,2159,SV Zulte Waregem,0,5,False,1,2
2844,2012,2,2012/08/05,18:00:00,2229917,2495,Oud-Heverlee Leuven,290,KRC Genk,2,2,False,1,3
3183,2011,15,2011/11/26,18:00:00,1134359,2159,SV Zulte Waregem,35,RSC Anderlecht,2,3,False,0,3


In [4]:
# controle van aantal_wedstrijden
# Geen enkel record met meer wedstrijden dan speeldagen
number_of_games_check = standings_df[standings_df.speeldag < standings_df.aantal_wedstrijden]
number_of_games_check.head(30)

,seizoen,speeldag,stand,naam_ploeg,id_ploeg,aantal_wedstrijden,aantal_gewonnen,aantal_gelijk,aantal_verloren,doelpunten_voor,doelpunten_tegen,verschil,punten


In [5]:
# controle van aantal_wedstrijden
# wel records met minder wedstrijden dan speeldagen, maar bij nazicht van een aantal records klopt dit
number_of_games_check = standings_df[standings_df.speeldag > standings_df.aantal_wedstrijden]
number_of_games_check.seizoen.value_counts()

1975    551
2002    476
2009    360
2023     16
Name: seizoen, dtype: int64

In [6]:
# geen records met standings_df.Played != (standings_df.Wins + standings_df.Losses + standings_df.Draws
number_of_games_check = standings_df[standings_df.aantal_wedstrijden != (standings_df.aantal_gewonnen + standings_df.aantal_gelijk + standings_df.aantal_verloren)]
number_of_games_check.count()

seizoen               0
speeldag              0
stand                 0
naam_ploeg            0
id_ploeg              0
aantal_wedstrijden    0
aantal_gewonnen       0
aantal_gelijk         0
aantal_verloren       0
doelpunten_voor       0
doelpunten_tegen      0
verschil              0
punten                0
dtype: int64

In [7]:
# geen records met (uitslagen.doelpunten_voor - uitslagen.doelpunten_tegen) != (uitslagen.verschil)
standings_df['calculated_goal_verschil'] = standings_df['doelpunten_voor'] - standings_df['doelpunten_tegen']
controle_aantal_wedstrijden = standings_df[standings_df['calculated_goal_verschil'] != standings_df['verschil']]
controle_aantal_wedstrijden.count()

seizoen                     0
speeldag                    0
stand                       0
naam_ploeg                  0
id_ploeg                    0
aantal_wedstrijden          0
aantal_gewonnen             0
aantal_gelijk               0
aantal_verloren             0
doelpunten_voor             0
doelpunten_tegen            0
verschil                    0
punten                      0
calculated_goal_verschil    0
dtype: int64

In [8]:
# het totaal aantal gewonnen matchen per seizoen en per ploeg
points_df = standings_df.groupby(['seizoen','speeldag','id_ploeg', 'punten'])[['aantal_gewonnen','aantal_gelijk','aantal_verloren']].sum()
points_df = points_df.reset_index()
points_df = points_df.sort_values(['seizoen','speeldag','aantal_gewonnen','aantal_gelijk','aantal_verloren'], ascending = [True, True, False, False, False])
points_df.head(30)

,seizoen,speeldag,id_ploeg,punten,aantal_gewonnen,aantal_gelijk,aantal_verloren
1,1960,1,2,2:0,1,0,0
3,1960,1,4,2:0,1,0,0
5,1960,1,10,2:0,1,0,0
6,1960,1,13,2:0,1,0,0
7,1960,1,16,2:0,1,0,0
10,1960,1,35,2:0,1,0,0
11,1960,1,90,2:0,1,0,0
14,1960,1,553,2:0,1,0,0
0,1960,1,1,0:2,0,0,1
2,1960,1,3,0:2,0,0,1


In [9]:
points_df['punten_check'] = points_df['aantal_gewonnen'] * 3 + points_df['aantal_gelijk'] + points_df['aantal_verloren'] * 0
discrepancies_df = points_df[points_df['punten'] != points_df['punten_check']]
discrepancies_df.head(30)

,seizoen,speeldag,id_ploeg,punten,aantal_gewonnen,aantal_gelijk,aantal_verloren,punten_check
1,1960,1,2,2:0,1,0,0,3
3,1960,1,4,2:0,1,0,0,3
5,1960,1,10,2:0,1,0,0,3
6,1960,1,13,2:0,1,0,0,3
7,1960,1,16,2:0,1,0,0,3
10,1960,1,35,2:0,1,0,0,3
11,1960,1,90,2:0,1,0,0,3
14,1960,1,553,2:0,1,0,0,3
0,1960,1,1,0:2,0,0,1,0
2,1960,1,3,0:2,0,0,1,0


In [10]:
match_results_df['datum'] = pd.to_datetime(match_results_df['datum'])
goal_events_df['datum'] = pd.to_datetime(goal_events_df['datum'])

merged_df = pd.merge(goal_events_df, match_results_df, on='id_match', suffixes=('_goal', '_match'))

# Check for discrepancies in dates
date_discrepancies_df = merged_df[merged_df['datum_goal'] != merged_df['datum_match']]
date_discrepancies_df.head(30)

,seizoen_goal,dag,datum_goal,tijd_goal,id_match,thuisploeg_stamnummer_goal,thuisploeg_goal,uitploeg_goal,uitploeg_stamnummer_goal,goal_time,...,speeldag,datum_match,tijd_match,thuisploeg_stamnummer_match,thuisploeg_match,uitploeg_stamnummer_match,uitploeg_match,doelpunten_thuisploeg,doelpunten_uitploeg,same_as_previous_match
2153,2021,22,2022-01-14,20:45:00,3598293,7,KAA Gent,KVK,100,8',...,22,2022-01-15,16:15:00,2495,Oud-Heverlee Leuven,25,KV Mechelen,0,0,True
2155,2021,22,2022-01-14,20:45:00,3598293,7,KAA Gent,KVK,100,44',...,22,2022-01-15,16:15:00,2495,Oud-Heverlee Leuven,25,KV Mechelen,0,0,True
2157,2021,22,2022-01-14,20:45:00,3598293,7,KAA Gent,KVK,100,52',...,22,2022-01-15,16:15:00,2495,Oud-Heverlee Leuven,25,KV Mechelen,0,0,True
2159,2021,22,2022-01-14,20:45:00,3598293,7,KAA Gent,KVK,100,90+1',...,22,2022-01-15,16:15:00,2495,Oud-Heverlee Leuven,25,KV Mechelen,0,0,True
2802,2020,12,2020-11-08,20:45:00,3415335,12,Cercle Brugge,Zulte,622,6',...,12,2020-12-08,17:00:00,2495,Oud-Heverlee Leuven,216,Royal Excel Moeskroen (-2022),0,0,True
2804,2020,12,2020-11-08,20:45:00,3415335,12,Cercle Brugge,Zulte,622,31',...,12,2020-12-08,17:00:00,2495,Oud-Heverlee Leuven,216,Royal Excel Moeskroen (-2022),0,0,True
2806,2020,12,2020-11-08,20:45:00,3415335,12,Cercle Brugge,Zulte,622,67',...,12,2020-12-08,17:00:00,2495,Oud-Heverlee Leuven,216,Royal Excel Moeskroen (-2022),0,0,True
2808,2020,12,2020-11-08,20:45:00,3415335,12,Cercle Brugge,Zulte,622,78',...,12,2020-12-08,17:00:00,2495,Oud-Heverlee Leuven,216,Royal Excel Moeskroen (-2022),0,0,True
2809,2020,12,2020-12-08,17:00:00,3415335,2495,OH Leuven,REM,190,56',...,12,2020-11-08,20:45:00,12,Cercle Brugge,2159,SV Zulte Waregem,1,3,False
2811,2020,12,2020-12-08,17:00:00,3415335,2495,OH Leuven,REM,190,84',...,12,2020-11-08,20:45:00,12,Cercle Brugge,2159,SV Zulte Waregem,1,3,False


In [11]:
# Groepeer de gegevens op 'Day' en controleer de datums
sorted_dates = match_results_df.sort_values(by=['speeldag', 'datum'])
date_discrepancies_df = sorted_dates.groupby('speeldag')['datum'].apply(lambda x: x.is_monotonic_increasing)

date_discrepancies_df = date_discrepancies_df[date_discrepancies_df == False]
date_discrepancies_df.head(30)


Series([], Name: datum, dtype: bool)

In [12]:
# Sorting the data by season, day, and date
sorted_dates = match_results_df.sort_values(by=['seizoen', 'speeldag', 'datum'])

# Group by 'season' and 'day', and check if dates are in chronological order within each group
grouped = sorted_dates.groupby(['seizoen', 'speeldag'])
date_order_check = grouped['datum'].apply(lambda x: x.is_monotonic_increasing)

# Identifying seasons and days where dates are not in chronological order
date_discrepancies = date_order_check[date_order_check == False]
date_discrepancies.head(30)

Series([], Name: datum, dtype: bool)

In [13]:
# Filter matches that ended 0-0
matches_0_0 = match_results_df[(match_results_df['doelpunten_thuisploeg'] == 0) & (match_results_df['doelpunten_uitploeg'] == 0)]

# Get match_ids of 0-0 matches
match_ids_0_0 = matches_0_0['id_match']

# Check if these matches have any goals in goal_events.csv
goals_in_0_0_matches = goal_events_df[goal_events_df['id_match'].isin(match_ids_0_0)]
goals_in_0_0_matches.head(30)

,seizoen,dag,datum,tijd,id_match,thuisploeg_stamnummer,thuisploeg,uitploeg,uitploeg_stamnummer,goal_time,goal_team_naam,goal_team_stamnummer,stand_thuis,stand_uit,valid_goal
2405,2021,22,2022-01-14,20:45:00,3598293,7,KAA Gent,KVK,100,8',KVK,100,0.0,1,True
2406,2021,22,2022-01-14,20:45:00,3598293,7,KAA Gent,KVK,100,44',KAA Gent,7,1.0,1,True
2407,2021,22,2022-01-14,20:45:00,3598293,7,KAA Gent,KVK,100,52',KAA Gent,7,2.0,1,True
2408,2021,22,2022-01-14,20:45:00,3598293,7,KAA Gent,KVK,100,90+1',KVK,100,2.0,2,True
3118,2020,12,2020-11-08,20:45:00,3415335,12,Cercle Brugge,Zulte,622,6',Zulte,622,0.0,1,True
3119,2020,12,2020-11-08,20:45:00,3415335,12,Cercle Brugge,Zulte,622,31',Zulte,622,0.0,2,True
3120,2020,12,2020-11-08,20:45:00,3415335,12,Cercle Brugge,Zulte,622,67',Cercle Brugge,12,1.0,2,True
3121,2020,12,2020-11-08,20:45:00,3415335,12,Cercle Brugge,Zulte,622,78',Zulte,622,1.0,3,True
3122,2020,12,2020-12-08,17:00:00,3415335,2495,OH Leuven,REM,190,56',OH Leuven,2495,1.0,0,True
3123,2020,12,2020-12-08,17:00:00,3415335,2495,OH Leuven,REM,190,84',OH Leuven,2495,2.0,0,True


In [14]:
# Sorteer de DataFrame op basis van seizoen, speeldag, punten, aantal gewonnen wedstrijden, doelpuntenverschil en gemaakte doelpunten
sorted_df = standings_df.sort_values(['seizoen', 'speeldag', 'punten', 'aantal_gewonnen', 'calculated_goal_verschil', 'doelpunten_voor'], ascending=[True, True, False, False, False, False])

# Bereken de stand
sorted_df['calculated_stand'] = sorted_df.groupby(['seizoen', 'speeldag']).cumcount() + 1

# Pas de stand aan voor gelijke standen
sorted_df['calculated_stand'] = sorted_df.groupby(['seizoen', 'speeldag', 'punten', 'aantal_gewonnen', 'calculated_goal_verschil', 'doelpunten_voor'])['calculated_stand'].transform('min')

# Voeg de berekende stand toe aan de originele DataFrame
standings_df = standings_df.merge(sorted_df[['seizoen', 'speeldag', 'id_ploeg', 'calculated_stand']], on=['seizoen', 'speeldag', 'id_ploeg'], how='left')

# Controleer op discrepanties tussen de berekende en werkelijke stand
discrepancies = standings_df[standings_df['stand'] != standings_df['calculated_stand']]
discrepancies.head(30)


,seizoen,speeldag,stand,naam_ploeg,id_ploeg,aantal_wedstrijden,aantal_gewonnen,aantal_gelijk,aantal_verloren,doelpunten_voor,doelpunten_tegen,verschil,punten,calculated_goal_verschil,calculated_stand
3,2023,1,3,St-Truidense VV,373,1,1,0,0,1,0,1,3,1,4
4,2023,1,3,Antwerp FC,1,1,1,0,0,1,0,1,3,1,4
5,2023,1,6,R Charleroi SC,22,1,0,1,0,1,1,0,1,0,8
6,2023,1,6,KV Mechelen,25,1,0,1,0,1,1,0,1,0,8
9,2023,1,6,Club Brugge,3,1,0,1,0,1,1,0,1,0,8
10,2023,1,6,OH Leuven,18,1,0,1,0,1,1,0,1,0,8
11,2023,1,12,Cercle Brugge,12,1,0,0,1,0,1,-1,0,-1,13
13,2023,1,12,Standard Luik,16,1,0,0,1,0,1,-1,0,-1,13
18,2023,2,2,St-Truidense VV,373,2,2,0,0,2,0,2,6,2,3
20,2023,2,4,Club Brugge,3,2,1,1,0,2,1,1,4,1,5
